In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import altair as alt
import calitp_data_analysis.magics
import pandas as pd

from calitp_data_analysis import calitp_color_palette as cp
from segment_speed_utils.project_vars import SEGMENT_GCS, SCHED_GCS
from shared_utils import rt_dates

analysis_date_list = rt_dates.y2024_dates + rt_dates.y2023_dates

alt.renderers.enable("html")
alt.data_transformers.enable('default', max_rows=None)

In [ ]:
#name = "Big Blue Bus Schedule"

In [ ]:
%%capture_parameters
name

In [ ]:
def merge_in_standardized_route_names(
    df: pd.DataFrame, 
    operator_name: str
) -> pd.DataFrame:
    standardized_route_names = pd.read_parquet(
        f"{SCHED_GCS}standardized_route_ids.parquet",
        columns = ["schedule_gtfs_dataset_key", "name", 
                   "route_id", "service_date",
                   "recent_route_id2", "recent_combined_name"
                  ],
        filters = [[("name", "==", name)]]
    )
    
    df = pd.merge(
        df,
        standardized_route_names,
        on = ["schedule_gtfs_dataset_key", "route_id", "service_date"],
        how = "left",
    )
    
    df = df.assign(
        route_short_name = (df.recent_combined_name
                            .str.split("__", expand=True)[0]),
        route_long_name = (df.recent_combined_name
                           .str.split("__", expand=True)[1]),
    ).drop(
        columns = ["route_id", "recent_combined_name"]
    ).rename(
        columns = {"recent_route_id2": "route_id"}
    )
    
    return df

In [ ]:
df = pd.concat([
    pd.read_parquet(
        f"{SEGMENT_GCS}rollup_singleday/speeds_route_dir_{d}.parquet",
        filters = [[("name", "==", name)]]
    ).drop(
        columns = "geometry"
    ).assign(
        service_date = pd.to_datetime(d)
    ) for d in analysis_date_list], 
    axis=0, ignore_index = True
).pipe(merge_in_standardized_route_names, name) 

In [ ]:
def base_chart(df: pd.DataFrame, y_col: str) -> alt.Chart:
    
    df = df.reset_index(drop=True)
    
    selected_colors = cp.CALITP_CATEGORY_BRIGHT_COLORS[3:]
    
    snakecase_ycol = y_col.replace('_', ' ')
    
    chart = (alt.Chart(df)
         .mark_bar(width=15)
         .encode(
             x = alt.X("yearmonthdate(service_date):O", title = "Date",
                       axis = alt.Axis(format = '%b %Y')
                      ),
             y = alt.Y(f"{y_col}:Q", title = snakecase_ycol,
                       #scale = alt.Scale(domain=[0, 50])
                      ),
             color = alt.Color(
                 "time_period:N", title = "",
                 scale = alt.Scale(range = selected_colors)
             ),
             tooltip = ["route_id", "direction_id", 
                        "time_period", y_col]
         ).facet(
             column = alt.Column("time_period:O", title="Time Period"),
             row = alt.Row("direction_id:N", title="Direction")
         ).interactive()
    )
    
    return chart

In [ ]:
def dual_chart(
    df, 
    control_field: str,
    y_col: str
):
    """
    https://stackoverflow.com/questions/58919888/multiple-selections-in-altair
    """
    route_dropdown = alt.binding_select(
        options=sorted(df[control_field].unique().tolist()), 
        name='Routes '
    )
        
    # Column that controls the bar charts
    route_selector = alt.selection_point(
        fields=[control_field], 
        bind=route_dropdown
    )
                
    chart = base_chart(
        df, y_col
    ).add_params(route_selector).transform_filter(route_selector)
   
    return chart

# {name}
## Route Average Speeds

Some of the routes have average speeds that jump wildly from month to month. Let's take a look at these.

In [ ]:
chart = dual_chart(
    df, 
    control_field = "route_id", 
    y_col = "speed_mph"
)

chart